In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import random
import math
import sqlite3
import pandas as pd
import numpy as np
import urllib
from lxml import etree
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import sqlite3
import pickle
import math
import numpy as np

In [ ]:
#initial request
url = 'https://www.basketball-reference.com/'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

In [ ]:
#find links for individual page
urls = []
for i in soup.find_all('tr', class_="full_table"):
    urls.append(url + i.find('a', href=True)['href'])
urls

In [ ]:
with open('listofurls.pkl', 'wb') as f:
    pickle.dump(urls, f)

In [ ]:
option = webdriver.ChromeOptions()
option.add_argument(" — incognito")
driver = webdriver.Chrome(executable_path='/Users/Anubhav/Desktop/Flatiron School/QCstats/chromedriver', options=option)

In [ ]:
all_individuals = pd.DataFrame()
all_teamstats = pd.DataFrame()
for url in range(len(urls)):
    #go to team site
    driver.get(urls[url])
    WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.ID, "all_roster")))

    #create dataframes for each table
    df0 = pd.read_html(driver.find_element_by_css_selector('table[id="per_game"]').get_attribute('outerHTML'))[0]
    df1 = pd.read_html(driver.find_element_by_css_selector('table[id="totals"]').get_attribute('outerHTML'))[0]
    df2 = pd.read_html(driver.find_element_by_css_selector('table[id="per_minute"]').get_attribute('outerHTML'))[0]
    df3 = pd.read_html(driver.find_element_by_css_selector('table[id="per_poss"]').get_attribute('outerHTML'))[0]
    df4 = pd.read_html(driver.find_element_by_css_selector('table[id="advanced"]').get_attribute('outerHTML'))[0]
    df5 = pd.read_html(driver.find_element_by_css_selector('table[id="shooting"]').get_attribute('outerHTML'))[0]
    df6 = pd.read_html(driver.find_element_by_css_selector('table[id="pbp"]').get_attribute('outerHTML'))[0] 
    df7 = pd.read_html(driver.find_element_by_css_selector('table[id="salaries2"]').get_attribute('outerHTML'))[0]
    df8 = pd.read_html(driver.find_element_by_css_selector('table[id="roster"]').get_attribute('outerHTML'))[0]

    #change column names of all player tables
    df0.columns = ['Rk', 'Name', 'Age', 'G', 'GS', 'MP/G', 'FGMade/G', 'FGA/G', 'FGPerc',
                   'ThreeMade/G', 'ThreeA/G', 'ThreePerc','TwoMade/G', 'TwoA/G',
                   'TwoPerc', 'eFGPerc', 'FTMade/G', 'FTA/G', 'FTPerc', 'ORB/G',
                   'DRB/G', 'TRB/G','AST/G', 'STL/G', 'BLK/G', 'TOV/G', 'PF/G', 'PTS/G']
    df1.columns = ['Rk', 'Name', 'Age', 'G', 'GS', 'MPTotal', 'FGMadeTotal',
                   'FGATotal', 'FGPerc', 'ThreeMadeTotal', 'ThreeATotal', 'ThreePerc',
                   'TwoMadeTotal', 'TwoATotal', 'TwoPerc', 'eFGPerc', 'FTMadeTotal',
                   'FTATotal', 'FTPerc', 'ORBTotal', 'DRBTotal', 'TRBTotal',
                   'ASTTotal', 'STLTotal', 'BLKTotal', 'TOVTotal', 'PFTotal', 'PTSTotal']
    df1.drop(df1.tail(1).index, inplace=True)
    df2.columns = ['Rk', 'Name', 'Age', 'G', 'GS', 'MP/THREESIX', 'FGMade/THREESIX',
                   'FGA/THREESIX', 'FGPerc', 'ThreeMade/THREESIX', 'ThreeA/THREESIX',
                   'ThreePerc','TwoMade/THREESIX', 'TwoA/THREESIX', 'TwoPerc',
                   'FTMade/THREESIX', 'FTA/THREESIX', 'FTPerc', 'ORB/THREESIX', 'DRB/THREESIX',
                   'TRB/THREESIX', 'AST/THREESIX', 'STL/THREESIX', 'BLK/THREESIX',
                   'TOV/THREESIX', 'PF/THREESIX', 'PTS/THREESIX']
    df3.columns = ['Rk', 'Name', 'Age', 'G', 'GS', 'MP/Hund', 'FGMade/Hund', 'FGA/Hund',
                   'FGPerc', 'ThreeMade/Hund', 'ThreeA/Hund', 'ThreePerc',
                   'TwoMade/Hund', 'TwoA/Hund', 'TwoPerc', 'FTMade/Hund', 'FTA/Hund',
                   'FTPerc', 'ORB/Hund', 'DRB/Hund', 'TRB/Hund', 'AST/Hund', 'STL/Hund',
                   'BLK/Hund', 'TOV/Hund', 'PF/Hund', 'PTS/Hund', 'Blank', 'ORTG', 'DRTG']
    df4.columns = ['Rk', 'Name', 'Age', 'G', 'MPTotal', 'PER', 'TSPerc', 'PercThreeA',
                   'PercFTA/FG', 'PercORB', 'PercDRB', 'PercTRB', 'PercAST', 'PercSTL',
                   'PercBLK', 'PercTOV', 'USG', 'Blank1', 'OWS', 'DWS', 'WS', 'WS/48',
                   'Blank2', 'OBPM', 'DBPM', 'BPM', 'VORP']
    df5.columns = ['Rk', 'Name', 'Age', 'G', 'MPTotal', 'FGPerc', 'AvgDis',
                   'PercTwoA', 'PercZeroThreefeetA','PercThreefeetTenA', 'PercTenSixTA',
                   'PercSixTThreeA', 'PercThreeA', 'TwoPerc', 'ZeroThreePerc',
                   'ThreefeetTenPerc', 'TenSixTPerc', 'SixTThreePerc', 'ThreePerc',
                   'PercTwoAstd', 'PercFGADunks', 'MadeDunks', 'PercThreeAstd',
                   'Perc3PACorner', 'ThreePercCorner', 'HeavesA', 'MadeHeaves']
    df6.columns = ['Rk', 'Name', 'Age', 'G', 'MP', 'PercasPG', 'PercasSG',
                   'PercasSF', 'PercasPF', 'PercasC', 'PMP100OnCourt', 'NPMP100',
                   'TOVBadPass', 'TOVLostBall', 'ShootFoulsC', 'OFoulsC',
                   'ShootFoulsD', 'OFoulsD', 'PGA', 'FAAnd1', 'FABlkd']
    df7.columns = ['Rk', 'Name', 'Salary']
    df7.Salary = df7.Salary.str.replace(',', '').str[1:]
    df7.drop_duplicates(subset='Name', keep='first',inplace=True)
    df8.columns = ['No', 'Name', 'Pos', 'Ht', 'Wt', 'Birth Date', 'Country', 'Exp', 'College']
    df8.Ht = (pd.to_numeric(df8.Ht.str.split('-').str[0])*12) + pd.to_numeric(df8.Ht.str.split('-').str[1])
    df8.drop(['No', 'Pos', 'Wt', 'Birth Date', 'Country', 'College'], axis = 1, inplace = True)
    df8.drop_duplicates(subset='Name', keep='first',inplace=True)
    
    for table in range(0,9):
        exec(f"df{table}.Name = df{table}.Name.str.split().str[0] + ' ' + df{table}.Name.str.split().str[1]")

    #drop uncessary columns from each dataframe
    for table in range(1,9):
        checklist = []
        for x in range(0,table):
            exec(f"checklist.extend(list(df{x}.columns))")
            checklist.remove('Name')
        exec(f"df{table}.drop([col for col in df{table}.columns if col in checklist], axis = 1, inplace = True)")
        exec(f"blank = [col for col in df{table} if col.startswith('Blank')]")
        if not blank:
            pass
        else:
            exec(f"df{table}.drop(blank, axis = 1, inplace = True)")

    #full outer join all dataframes
    for x in range(0,9):
        exec(f"df{x}.set_index('Name', inplace = True)")
        
    namesnotindf7index = [x for x in df7.index if x not in df0.index]
    namesnotindf8index = [x for x in df8.index if x not in df0.index]
    df7.drop(namesnotindf7index, axis=0, inplace=True)
    df8.drop(namesnotindf8index, axis=0, inplace=True)
    
    result = pd.concat([df0, df1, df2, df3, df4, df5, df6, df7, df8], axis=1, sort = True)
    result.drop('Rk', axis = 1, inplace = True)
    result.reset_index(inplace = True)
    result.rename(columns = {'index': 'Name'}, inplace = True)

    #add TeamName column
    for idx, x in enumerate(result.Name):
        result.loc[idx, 'Team'] = urls[url][44:47]
    
    #add rows to pre-existing dataframe
    all_individuals = pd.concat([all_individuals, result], ignore_index=True)
    
    
    teamstats = pd.read_html(driver.find_element_by_css_selector('table[id="team_and_opponent"]').get_attribute('outerHTML'))[0]

    if 'FG.1' in teamstats.columns:
        ending = '.1'
    else:
        ending = '%'
        
    row = teamstats[teamstats.index == 0].drop('Unnamed: 0', axis=1)
    row.reset_index(drop=True, inplace=True)
    row1 = teamstats[teamstats.index == 1].drop(['Unnamed: 0', 'G', 'FG' + ending, '3P' + ending, '2P' + ending, 'FT' + ending], axis=1)
    row1.reset_index(drop=True, inplace=True)
    row2 = teamstats[teamstats.index == 4].drop('Unnamed: 0', axis=1)
    row2.reset_index(drop=True, inplace=True)
    row3 = teamstats[teamstats.index == 5].drop(['Unnamed: 0', 'G', 'FG' + ending, '3P' + ending, '2P' + ending, 'FT' + ending], axis=1)
    row3.reset_index(drop=True, inplace=True)
    row4 = teamstats[teamstats.index == 3].drop(['Unnamed: 0', 'G'], axis=1)
    row4.reset_index(drop=True, inplace=True)
    row5 = teamstats[teamstats.index == 4].drop(['Unnamed: 0', 'G'], axis=1)
    row5.reset_index(drop=True, inplace=True)

    row.columns = ['G', 'MPTotal', 'FGMadeTotal', 'FGATotal', 'FGPerc', 'ThreeMadeTotal', 'ThreeATotal',
     'ThreePerc', 'TwoMadeTotal', 'TwoATotal', 'TwoPerc', 'FTMadeTotal', 'FTATotal',
     'FTPerc', 'ORBTotal', 'DRBTotal', 'TRBTotal', 'ASTTotal', 'STLTotal','BLKTotal',
     'TOVTotal', 'PFTotal', 'PTSTotal']

    row1.columns = ['MP/G', 'FGMade/G', 'FGA/G', 'ThreeMade/G', 'ThreeA/G',
                     'TwoMade/G', 'TwoA/G', 'FTMade/G', 'FTA/G', 'ORB/G',
                     'DRB/G', 'TRB/G', 'AST/G', 'STL/G','BLK/G', 'TOV/G', 'PF/G', 'PTS/G']

    row2.columns = ['OppG', 'OppMPTotal', 'OppFGMadeTotal', 'OppFGATotal', 'OppFGPerc',
                     'OppThreeMadeTotal', 'OppThreeATotal', 'OppThreePerc', 'OppTwoMadeTotal',
                     'OppTwoATotal', 'OppTwoPerc', 'OppFTMadeTotal', 'OppFTATotal', 'OppFTPerc',
                     'OppORBTotal', 'OppDRBTotal', 'OppTRBTotal', 'OppASTTotal', 'OppSTLTotal',
                     'OppBLKTotal', 'OppTOVTotal', 'OppPFTotal', 'OppPTSTotal']

    row3.columns = ['OppMP/G', 'OppFGMade/G', 'OppFGA/G', 'OppThreeMade/G', 'OppThreeA/G',
                     'OppTwoMade/G', 'OppTwoA/G', 'OppFTMade/G', 'OppFTA/G', 'OppORB/G',
                     'OppDRB/G', 'OppTRB/G', 'OppAST/G', 'OppSTL/G','OppBLK/G', 'OppTOV/G',
                     'OppPF/G', 'OppPTS/G']

    row4.columns = ['MPDiff', 'FGMadeDiff', 'FGADiff', 'FGDiff', 'ThreeMadeDiff', 'ThreeADiff',
                     'ThreePercDiff', 'TwoMadeDiff', 'TwoADiff', 'TwoDiff', 'FTMadeDiff', 'FTADiff',
                     'FTPercDiff', 'ORBDiff', 'DRBDiff', 'TRBDiff', 'ASTDiff', 'STLDiff','BLKDiff',
                     'TOVDiff', 'PFDiff', 'PTSDiff']

    row5.columns = ['OppMPDiff', 'OppFGMadeDiff', 'OppFGADiff', 'OppFGDiff', 'OppThreeMadeDiff', 'OppThreeADiff',
                     'OppThreePercDiff', 'OppTwoMadeDiff', 'OppTwoADiff', 'OppTwoDiff', 'OppFTMadeDiff', 'OppFTADiff',
                     'OppFTPercDiff', 'OppORBDiff', 'OppDRBDiff', 'OppTRBDiff', 'OppASTDiff', 'OppSTLDiff','OppBLKDiff',
                     'OppTOVDiff', 'OppPFDiff', 'OppPTSDiff']

    misc = pd.read_html(driver.find_element_by_css_selector('table[id="team_misc"]').get_attribute('outerHTML'))[0]

    misc.columns = ['Label', 'W', 'L', 'PW', 'PL', 'MOV', 'SOS',
                    'SRS', 'ORtg', 'DRtg', 'Pace', 'FTr', '3PAr',
                    'OeFGPerc', 'OTOVPerc', 'ORBPerc', 'OFT/FGA', 'DeFGPerc',
                    'DTOVPerc', 'DRBPerc', 'DFT/FGA', 'Arena', 'Attendance']

    row6 = misc[misc.index == 0].drop(['Label', 'Arena', 'Attendance'], axis=1)
    row6.reset_index(drop=True, inplace=True)

    teamstatsrow = pd.concat([row, row1, row2, row3, row4, row5, row6], axis=1, sort=True)
    teamstatsrow.loc[0, 'Team'] = urls[url][44:47]
    
    all_teamstats = pd.concat([all_teamstats, teamstatsrow], ignore_index=True)
    all_teamstats.reset_index(drop=True, inplace=True)

In [ ]:
all_individuals.to_pickle("individualstats.pkl")
all_teamstats.to_pickle("1920teamstats.pkl")